In [55]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np
import h5py

#  Import and read the data
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [56]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [57]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [58]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [59]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [60]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [61]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts_df = pd.DataFrame(class_counts)
class_counts_over_1 = class_counts_df.loc[(class_counts_df['CLASSIFICATION'] > 1)]
print(class_counts_over_1.value_counts)

<bound method DataFrame.value_counts of        CLASSIFICATION
C1000           17326
C2000            6074
C1200            4837
C3000            1918
C2100            1883
C7000             777
C1700             287
C4000             194
C5000             116
C1270             114
C2700             104
C2800              95
C7100              75
C1300              58
C1280              50
C1230              36
C1400              34
C7200              32
C2300              32
C1240              30
C8000              20
C7120              18
C1500              16
C1800              15
C6000              15
C1250              14
C8200              11
C1238              10
C1278              10
C1235               9
C1237               9
C7210               7
C2400               6
C1720               6
C4100               6
C1257               5
C1600               5
C1260               3
C2710               3
C0                  3
C3200               2
C1234               2
C1246         

In [62]:
# Used cutoff value of 1000 and made index into a column to grab classifications to be replaced by Other
class_counts_cutoff = class_counts_df.loc[(class_counts_df['CLASSIFICATION'] < 1000)]
class_counts_cutoff_new = class_counts_cutoff.reset_index()
class_counts_cutoff_new.rename(columns = {'index': 'class_names'}, inplace=True)
class_counts_cutoff_new.head()

,class_names,CLASSIFICATION
0,C7000,777
1,C1700,287
2,C4000,194
3,C5000,116
4,C1270,114


In [63]:
# Converting the classifications to be replaced into a list
classes_replace = class_counts_cutoff_new.class_names.tolist()

In [64]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classes_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [65]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(application_df[["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE",
"ORGANIZATION", "SPECIAL_CONSIDERATIONS"]])
dummies.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,1,1,0,0,0,1,0
3,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,1,0
4,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0


In [66]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = dummies

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [67]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [68]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=300, activation="relu", input_dim=32))

# Second layer
nn.add(tf.keras.layers.Dense(units=200, activation="relu"))

# Third layer
nn.add(tf.keras.layers.Dense(units=100, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 300)               9900      
                                                                 
 dense_13 (Dense)            (None, 200)               60200     
                                                                 
 dense_14 (Dense)            (None, 100)               20100     
                                                                 
 dense_15 (Dense)            (None, 1)                 101       
                                                                 
Total params: 90,301
Trainable params: 90,301
Non-trainable params: 0
_________________________________________________________________


In [69]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [70]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20
804/804 [==============================] - 4s 4ms/step - loss: 0.5735 - accuracy: 0.7174
Epoch 2/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5645 - accuracy: 0.7248
Epoch 3/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5625 - accuracy: 0.7243
Epoch 4/20
804/804 [==============================] - 4s 5ms/step - loss: 0.5616 - accuracy: 0.7255
Epoch 5/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5605 - accuracy: 0.7263
Epoch 6/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5605 - accuracy: 0.7274
Epoch 7/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5591 - accuracy: 0.7265
Epoch 8/20
804/804 [==============================] - 4s 5ms/step - loss: 0.5583 - accuracy: 0.7282
Epoch 9/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5581 - accuracy: 0.7284
Epoch 10/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5586 - accuracy: 0.7269

In [71]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5608 - accuracy: 0.7254 - 564ms/epoch - 2ms/step
Loss: 0.560840904712677, Accuracy: 0.7253644466400146


In [72]:
# Export our model to HDF5 file
# wasn't sure about this step - haven't discussed HDF5 files in class
nn.save('AlphabetSoupCharity_Optimization_3.h5')